In [1]:
import cv2
import numpy as np

In [2]:
from ultralytics import YOLO

In [3]:
model_pose=YOLO('yolov8\yolov8x-pose-p6.pt')

In [4]:
class violence:
  def pose(self,frame):
    b=np.zeros_like(frame)
    res=model_pose(frame)
    plt=res[0].plot(img=b,labels=False, boxes=False, masks=False, probs=False)
    return plt
  def change(self,prev,curr):
    prev_gray = cv2.cvtColor(prev,cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(prev)
    mask[..., 1] = 255
    gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray,None,0.5, 3, 15, 3, 5, 1.2, 0)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    mask[..., 0] = angle * 180 / np.pi / 2
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    return rgb
  def fusion(self,rgb,cd):
    img = cv2.addWeighted(rgb, 0.4, cd, 0.6, 0)
    return img

In [5]:
import tensorflow as tf

In [6]:
new_model = tf.keras.models.load_model('result/violence.h5')

In [7]:
vgg_model = tf.keras.applications.VGG16(include_top=True, weights='imagenet')


In [8]:
transfer_layer = vgg_model.get_layer('fc2')

In [9]:
vgg_model_final = tf.keras.models.Model(inputs=vgg_model.input,
                             outputs=transfer_layer.output)

In [11]:
import pyfirmata
import time
port = 'COM3' 
board = pyfirmata.Arduino(port)
board.digital[11].mode=pyfirmata.PWM
alarm_pin = board.get_pin('d:11:o')
def trigger_alarm():
    alarm_pin.write(1)  
    time.sleep(2)
    alarm_pin.write(0) 
    time.sleep(1) 

In [18]:
cap=cv2.VideoCapture('fi87_xvid.avi')
v=violence()
count=1
res1=[]
while True:
    print(count)
    ret,frame=cap.read()
    prev=np.zeros_like(frame)
    if ret:
        pose=v.pose(frame)
        opt=v.change(prev,frame)
        fus=v.fusion(pose,opt)
        fus=fus[32:-32,68:-68]
        fus=fus.reshape(1,224,224,3)
        fus=vgg_model_final(fus)
        res1.append(fus[0])
        count+=1
        cv2.imshow('video',frame)
        if count>=21:
            res1=np.array(res1)
            res1=np.array(res1).reshape(1,20,4096)
            res=new_model.predict(res1)
            res1=res1[0,1:,:]
            res1 = res1.tolist()
            if res[0][0]>0.9906:
                print('violence')
                #for i in range(5):
                #    trigger_alarm()
            else:
                print("Non-violence")
    else:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
    prev=frame
cap.release() 
cv2.destroyAllWindows() 

1


0: 1024x1280 3 persons, 15259.3ms
Speed: 30.2ms preprocess, 15259.3ms inference, 3.0ms postprocess per image at shape (1, 3, 1024, 1280)



2


0: 1024x1280 2 persons, 15169.1ms
Speed: 35.7ms preprocess, 15169.1ms inference, 2.2ms postprocess per image at shape (1, 3, 1024, 1280)



3


...

0: 1024x1280 3 persons, 7397.3ms
Speed: 12.5ms preprocess, 7397.3ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 1280)


1/1 [==============================] - 0s 52ms/step


violence
41


0: 1024x1280 3 persons, 7362.7ms
Speed: 12.4ms preprocess, 7362.7ms inference, 4.0ms postprocess per image at shape (1, 3, 1024, 1280)


1/1 [==============================] - 0s 57ms/step
violence
42


In [20]:
cap=cv2.VideoCapture('no471_xvid.avi')
v=violence()
count=1
res1=[]
while True:
    print(count)
    ret,frame=cap.read()
    prev=np.zeros_like(frame)
    if ret:
        pose=v.pose(frame)
        opt=v.change(prev,frame)
        fus=v.fusion(pose,opt)
        fus=fus[32:-32,68:-68]
        #fus=fus[128:-128,208:-208]
        fus=fus.reshape(1,224,224,3)
        fus=vgg_model_final(fus)
        res1.append(fus[0])
        count+=1
        cv2.imshow('video',frame)
        if count>=21:
            res1=np.array(res1)
            res1=np.array(res1).reshape(1,20,4096)
            res=new_model.predict(res1)
            res1=res1[0,1:,:]
            res1 = res1.tolist()
            if res[0][0]>0.9906:
                print('violence')
                for i in range(5):
                    trigger_alarm()
            else:
                print("Non-violence")
    else:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
    prev=frame
cap.release() 
cv2.destroyAllWindows() 

1


0: 1024x1280 5 persons, 7054.6ms
Speed: 15.2ms preprocess, 7054.6ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 1280)



2


0: 1024x1280 5 persons, 7276.5ms
Speed: 13.5ms preprocess, 7276.5ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 1280)



3


...

0: 1024x1280 4 persons, 7733.3ms
Speed: 14.1ms preprocess, 7733.3ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 1280)


1/1 [==============================] - 0s 50ms/step


Non-violence
41


0: 1024x1280 4 persons, 8201.5ms
Speed: 17.0ms preprocess, 8201.5ms inference, 2.5ms postprocess per image at shape (1, 3, 1024, 1280)


1/1 [==============================] - 0s 59ms/step
Non-violence
42


In [14]:
cap=cv2.VideoCapture(0)
v=violence()
count=1
res1=[]
while True:
    print(count)
    ret,frame=cap.read()
    prev=np.zeros_like(frame)
    if ret:
        pose=v.pose(frame)
        opt=v.change(prev,frame)
        fus=v.fusion(pose,opt)
        fus=fus[128:-128,208:-208]
        print(fus.shape)
        fus=fus.reshape(1,224,224,3)
        fus=vgg_model_final(fus)
        print(fus.shape)
        res1.append(fus[0])
        count+=1
        cv2.imshow('video',frame)
        if count>=21:
            res1=np.array(res1)
            print(res1.shape)
            res1=np.array(res1).reshape(1,20,4096)
            res=new_model.predict(res1)
            res1=res1[0,1:,:]
            res1 = res1.tolist()
            print(res)
            if res[0][0]>0.9907:
                print('violence')
                print(res)
                for i in range(5):
                     trigger_alarm()
            else:
                print("Non-violence")
    else:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
    prev=frame
cap.release() 
cv2.destroyAllWindows() 

1



0: 960x1280 1 person, 15977.1ms
Speed: 39.1ms preprocess, 15977.1ms inference, 0.0ms postprocess per image at shape (1, 3, 960, 1280)


(224, 224, 3)


(1, 4096)
2


0: 960x1280 1 person, 16299.8ms
Speed: 21.9ms preprocess, 16299.8ms inference, 5.0ms postprocess per image at shape (1, 3, 960, 1280)


...

(224, 224, 3)
(1, 4096)
(20, 4096)
1/1 [==============================] - 0s 91ms/step


[[    0.99061   0.0093873]]
Non-violence
58


0: 960x1280 1 person, 13747.0ms
Speed: 19.7ms preprocess, 13747.0ms inference, 4.0ms postprocess per image at shape (1, 3, 960, 1280)


(224, 224, 3)
(1, 4096)
(20, 4096)
1/1 [==============================] - 0s 80ms/step
[[    0.99092   0.0090838]]
violence
[[    0.99092   0.0090838]]


SerialException: WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))